In [1]:
from pyspark import SparkConf, SparkContext
from collections import OrderedDict
from functools import partial

In [ ]:
parcsv = sc.textFile("/home/milroy/pyspark/2014-04-24.csv", 12).map(
    lambda line: (line.split(';')[0], line.split(';')[1], 
                  line.split(';')[2], line.split(';')[5]))
    
columns = parcsv.map(lambda x: str(x[1])).distinct().collect()
basedict = {col:0.0 for col in columns}
ordered = OrderedDict(sorted(basedict.items(), key=lambda t: t[0]))

def cmbPivot(u1, u2):
  u1.update(u2)
  return u1

def seqPivot(u, v):
  if not u:
      u = {}
  u[v[1]] = v[2]
  return u

def mergedicts(new, canon):
  tmp = canon.copy()
  tmp.update(new[1])
  return new[0], tmp

pivot = parcsv.keyBy(lambda row: (row[3], row[0])).aggregateByKey(None, seqPivot, cmbPivot)

final = pivot.map(partial(mergedicts, canon=ordered)).take(100)
